### Chatbot based on Seq2Seq Beam Search + Attention + Reinforcment Learning(Experimental)
- Tensorflow 1.4.0+ is required.
- This is based on [NMT Tutorial](https://github.com/tensorflow/nmt).
- Experiment [notes](https://github.com/higepon/tensorflow_seq2seq_chatbot/wiki).



In [0]:
# Special commands should be located here.
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
!apt-get -qq install -y mecab libmecab-dev mecab-ipadic mecab-ipadic-utf8

!pip -q install git+https://github.com/mrahtz/easy-tf-log#egg=easy-tf-log[tf]
!pip install pushbullet.py
!pip install tweepy pyyaml
!pip install mecab-python3

def auth_google_drive():
  # Generate creds for the Drive FUSE library.
  if not os.path.exists('drive'):
    from oauth2client.client import GoogleCredentials
    creds = GoogleCredentials.get_application_default()
    import getpass
    !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
    vcode = getpass.getpass()
    !echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}  

def mount_google_drive():
  if not os.path.exists('drive'):
    os.makedirs('drive', exist_ok=True)
    !google-drive-ocamlfuse drive 
    
def kill_docker():
  !kill -9 -1  


In [0]:
import urllib.request
response = urllib.request.urlopen("https://raw.githubusercontent.com/yaroslavvb/memory_util/master/memory_util.py")
open("memory_util.py", "wb").write(response.read())
import memory_util

In [0]:
from __future__ import print_function

import copy as copy
import datetime
import hashlib
import json
import os
import os.path
import filecmp
import random
import re
import shutil
import importlib


import MeCab
import easy_tf_log
import matplotlib.pyplot as plt
import random as random
import numpy as np
import tensorflow as tf
import tweepy
import yaml
from easy_tf_log import tflog
from google.colab import auth
from google.colab import files
import importlib
from pushbullet import Pushbullet
from tensorflow.python.layers import core as layers_core
from tensorflow.python.platform import gfile

# Generate auth tokens for Colab
auth.authenticate_user()


In [0]:
#kill_docker()

In [0]:
auth_google_drive()
mount_google_drive()

In [0]:
import drive.tensorflow_seq2seq_chatbot.lib.chatbot_model as sq

In [0]:
def reload_modules():
  !fusermount -u drive
  !google-drive-ocamlfuse -cc drive 
  importlib.reload(sq)


In [0]:
if sq.mode == sq.Mode.Test:
    sq.test_distributed_one(enable_attention=False)
    sq.test_distributed_one(enable_attention=True)


In [0]:
tweet_small_hparams = copy.deepcopy(sq.base_hparams).override_from_dict(
    {
        'batch_size': 6,  # of tweets should be dividable by batch_size
        'encoder_length': 8,
        'decoder_length': 8,
        'num_units': 256,
        'num_layers': 2,
        'vocab_size': 34,
        'embedding_size': 40,
        'beam_width': 2,  # for faster iteration, this should be 10
        'num_train_steps': 200,
        'model_path': sq.ModelDirectory.tweet_small.value,
        'learning_rate': 0.05,
        'use_attention': True,
    })

tweet_small_swapped_hparams = copy.deepcopy(
    tweet_small_hparams).override_from_dict(
    {'model_path': sq.ModelDirectory.tweet_small_swapped.value})

if sq.mode == sq.Mode.Test:
    tweets_path = "tweets_small.txt"
    sq.TrainDataGenerator(tweets_path, tweet_small_hparams).remove_generated()
    trainer = sq.Trainer()
    trainer.train_seq2seq(tweet_small_hparams, tweets_path,
                          ["おはようございます。寒いですね。", "さて帰ろう。明日は早い。", "今回もよろしくです。"])
    sq.test_tweets_small_swapped(tweet_small_swapped_hparams)


In [0]:
tweet_large_hparams = copy.deepcopy(sq.base_hparams).override_from_dict(
    {
        # In typical seq2seq chatbot
        # num_layers=3, learning_rate=0.5, batch_size=64, vocab=20000-100000, learning_rate decay is 0.99, which is taken care as default parameter in AdamOptimizer.
        'batch_size': 64,  # of tweets should be dividable by batch_size
        'encoder_length': 28,
        'decoder_length': 28,
        'num_units': 1024,
        'num_layers': 3,
        'vocab_size': 60000,
    # conversations.txt actually has about 70K uniq words.
        'embedding_size': 1024,
        'beam_width': 2,  # for faster iteration, this should be 10
        'num_train_steps': 1000000,
        'model_path': sq.ModelDirectory.tweet_large.value,
        'learning_rate': 0.5,
    # For vocab_size 50000, num_layers 3, num_units 1024, tweet_large, starting learning_rate 0.05 works well, change it t0 0.01 at perplexity 800, changed it to 0.005 at 200.
        'learning_rate_decay': 0.99,
        'use_attention': True,
        # testing new restore learning rate and no USERNAME TOKEN
    })

tweet_large_swapped_hparams = copy.deepcopy(
    tweet_large_hparams).override_from_dict(
    {
        'model_path': sq.ModelDirectory.tweet_large_swapped.value
    })

#Shell.save_model_in_drive(tweet_large_hparams.model_path)

if sq.mode == sq.Mode.TrainSeq2Seq:
    print("train seq2seq")
    sq.test_tweets_large(tweet_large_hparams)
elif sq.mode == sq.Mode.TrainSeq2SeqSwapped:
    print("train seq2seq swapped")
    sq.test_tweets_large_swapped(tweet_large_swapped_hparams)


In [0]:
!ls -Sl model/conversations_large*

In [0]:
  
reload_modules()

conversations_large_hparams = copy.deepcopy(sq.base_hparams).override_from_dict(
    {
        # In typical seq2seq chatbot
        # num_layers=3, learning_rate=0.5, batch_size=64, vocab=20000-100000, learning_rate decay is 0.99, which is taken care as default parameter in AdamOptimizer.
        'batch_size': 64,  # of tweets should be dividable by batch_size
        'encoder_length': 28,
        'decoder_length': 28,
        'num_units': 1024,
        'num_layers': 3,
        'vocab_size': 60000,
    # conversations.txt actually has about 70K uniq words.
        'embedding_size': 1024,
        'beam_width': 2,  # for faster iteration, this should be 10
        'num_train_steps': 0,
        'model_path': sq.ModelDirectory.conversations_large.value,
        'learning_rate': 0.5,
    # For vocab_size 50000, num_layers 3, num_units 1024, tweet_large, starting learning_rate 0.05 works well, change it t0 0.01 at perplexity 800, changed it to 0.005 at 200.
        'learning_rate_decay': 0.99,
        'use_attention': True,

    })

conversations_large_rl_hparams = copy.deepcopy(
    conversations_large_hparams).override_from_dict(
    {
        'model_path': sq.ModelDirectory.conversations_large_rl.value,
        'num_train_steps': 12000,
        'beam_width': 3,
    })


conversations_large_backward_hparams = copy.deepcopy(
    conversations_large_hparams).override_from_dict(
    {
        'model_path': sq.ModelDirectory.conversations_large_backward.value,
        'num_train_steps': 0,        
    })


conversations_txt = "conversations_large.txt"
sq.Shell.download_file_if_necessary(conversations_txt)
sq.ConversationTrainDataGenerator().generate(conversations_txt)

with memory_util.capture_stderr() as stderr:
    try:
        trainer =sq.Trainer()
        valid_tweets = ["さて福岡行ってきます！", "誰か飲みに行こう", "熱でてるけど、でもなんか食べなきゃーと思ってアイス買おうとしたの",
              "今日のドラマ面白そう！", "お腹すいたー", "おやすみ～", "おはようございます。寒いですね。",
              "さて帰ろう。明日は早い。", "今回もよろしくです。", "ばいとおわ！"]
        trainer.train_seq2seq(conversations_large_hparams,
                              "conversations_large_seq2seq.txt",
                              valid_tweets, should_clean_saved_model=False)
        trainer.train_seq2seq_swapped(conversations_large_backward_hparams,
                                      "conversations_large_seq2seq.txt",
                                      ["この難にでも応用可能なひどいやつ", "おはようございます。明日はよろしくおねがいします。"], vocab_path="conversations_large_seq2seq_vocab.txt", should_clean_saved_model=False)

        sq.Shell.copy_saved_model(conversations_large_hparams, conversations_large_rl_hparams)
        sq.Trainer().train_rl(conversations_large_rl_hparams,
                                conversations_large_hparams,
                                conversations_large_backward_hparams,
                                "conversations_large_seq2seq.txt",

                                "conversations_large_rl.txt",
                                valid_tweets)
    except Exception as e:
        print(stderr.getvalue())
        raise (e)

!ls - lSh



In [0]:
# N.B: This would fail if we try to download logs in the previous cell.
# My guess is tflog is somehow locking the log file when running the cell.
#download_logs()